# Data Pre-processing

In [14]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import numpy as np

class ThermalHistoryDataset(Dataset):
    def __init__(self, input_dir, output_dir, window_size=500, step_size=250, scaler=None):
        self.input_files = {os.path.basename(f): os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.txt')}
        self.output_files = {os.path.basename(f): os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.txt')}
        
        assert self.input_files.keys() == self.output_files.keys(), "Mismatch in input and output file names."
        
        self.file_names = sorted(self.input_files.keys())
        self.scaler = scaler or StandardScaler()
        self.window_size = window_size
        self.step_size = step_size
        self._fit_scaler()
        print(f"Initialized dataset with {len(self.file_names)} files.")

    def _fit_scaler(self):
        all_input_features = []
        for file_name in self.file_names:
            data = pd.read_csv(self.input_files[file_name], delimiter=',')[[ 'tm', 'tl', 'cr']].values
            all_input_features.append(data)
        all_input_features = np.vstack(all_input_features)
        self.scaler.fit(all_input_features)
        print("Scaler fitted to the data.")

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        input_data = pd.read_csv(self.input_files[file_name], delimiter=',')
        output_data = pd.read_csv(self.output_files[file_name], delimiter=',')

        input_coords = input_data[['x', 'y', 'z']].values
        input_features = input_data[['tm', 'tl', 'cr']].values
        output_features = output_data[['tm', 'tl', 'cr']].values

        input_features = self.scaler.transform(input_features)
        output_features = self.scaler.transform(output_features)

        input_features = np.concatenate((input_coords, input_features), axis=1)
        output_features = np.concatenate((input_coords, output_features), axis=1)

        input_windows = []
        output_windows = []

        for start in range(0, len(input_features) - self.window_size + 1, self.step_size):
            end = start + self.window_size
            input_windows.append(input_features[start:end])
            output_windows.append(output_features[start:end])

        input_windows = np.array(input_windows)
        output_windows = np.array(output_windows)

        if idx % 10 == 0:
            print(f"Processed file {idx + 1}/{len(self.file_names)}")

        return {
            'input_windows': torch.tensor(input_windows, dtype=torch.float),
            'output_windows': torch.tensor(output_windows, dtype=torch.float)
        }

def collate_fn(batch):
    input_windows = [item['input_windows'] for item in batch]
    output_windows = [item['output_windows'] for item in batch]

    input_windows = torch.cat(input_windows, dim=0)
    output_windows = torch.cat(output_windows, dim=0)

    return {
        'input_windows': input_windows,
        'output_windows': output_windows
    }

from torch.utils.data import DataLoader

# Create dataset with sliding window parameters
window_size = 250
step_size = 250
dataset = ThermalHistoryDataset(input_dir='./../../data/input', output_dir='./../../data/output', window_size=window_size, step_size=step_size)

# Create dataloader with custom collate function
batch_size = 8
num_workers = 4
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, num_workers=num_workers)


Scaler fitted to the data.
Initialized dataset with 3 files.


# Transformer Model

In [18]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        seq_len = x.size(1)
        device = x.device
        return x + self.encoding[:, :seq_len, :].to(device)

class SimpleTransformerModel(nn.Module):
    def __init__(self, feature_size, num_layers=1, num_heads=1, dim_feedforward=64, dropout_rate=0.1):
        super(SimpleTransformerModel, self).__init__()
        self.positional_encoding = PositionalEncoding(feature_size, max_len=500)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=feature_size,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            batch_first=True,
            dropout=dropout_rate
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.decoder = nn.Linear(feature_size, feature_size)

    def forward(self, src):
        print(f"Input tensor shape: {src.shape}, device: {src.device}")
        src = self.positional_encoding(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

# Initialize the model with simpler parameters
feature_size = 6
num_heads = 1
num_layers = 1
dim_feedforward = 64
dropout_rate = 0.1
model = SimpleTransformerModel(feature_size=feature_size, num_layers=num_layers, num_heads=num_heads, dim_feedforward=dim_feedforward, dropout_rate=dropout_rate)

# Move model to GPU
model = model.to('cuda')


# Training and Validation

In [21]:
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss Function
criterion = torch.nn.MSELoss()

# Initialize GradScaler for mixed precision training
scaler = GradScaler()

# Training Function with Gradient Accumulation and Mixed Precision
def train(model, dataloader, optimizer, criterion, scaler, accumulation_steps=4):
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(dataloader):
        input_windows = batch['input_windows'].to('cuda')
        output_windows = batch['output_windows'].to('cuda')
        
        print(f"Batch {batch_idx + 1} input_windows shape: {input_windows.shape}")
        print(f"Batch {batch_idx + 1} output_windows shape: {output_windows.shape}")
        
        with autocast():
            predictions = model(input_windows)
            loss = criterion(predictions, output_windows)
        
        scaler.scale(loss).backward()
        
        if (batch_idx + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        total_loss += loss.item()
        
        if batch_idx % 10 == 0:
            print(f"Processed batch {batch_idx + 1}/{len(dataloader)}")

    return total_loss / len(dataloader)

# Validation Function with Mixed Precision
def validate(model, dataloader, criterion, scaler):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            input_windows = batch['input_windows'].to('cuda')
            output_windows = batch['output_windows'].to('cuda')
            
            with autocast():
                predictions = model(input_windows)
                loss = criterion(predictions, output_windows)
                
            total_loss += loss.item()
            
            if batch_idx % 10 == 0:
                print(f"Processed validation batch {batch_idx + 1}/{len(dataloader)}")

    return total_loss / len(dataloader)

# Example Training Loop
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Reduce batch size significantly
batch_size = 2
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, num_workers=num_workers)

num_epochs = 10
for epoch in range(num_epochs):
    print(f"Starting epoch {epoch + 1}")
    train_loss = train(model, dataloader, optimizer, criterion, scaler)
    print(f"Epoch {epoch + 1}: Train Loss = {train_loss:.4f}")


Starting epoch 1
Processed file 1/3
Batch 1 input_windows shape: torch.Size([93396, 250, 6])
Batch 1 output_windows shape: torch.Size([93396, 250, 6])
Input tensor shape: torch.Size([93396, 250, 6]), device: cuda:0


RuntimeError: CUDA error: invalid configuration argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
